In [5]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pathlib

## Preliminary Analysis

In [8]:
images = pd.read_csv("../data/tiny-imagenet-200/words.txt", names = ['Id', 'labels'], sep = '\t')

In [3]:
#takes a while to run
images_expanded = pd.DataFrame(columns = ['Id', 'label'])
for img in images['Id']: 
    classes = images[images['Id'] == img]['labels']
    
    for img_class in classes: 
        images_expanded = images_expanded.append({'Id': img, 'label' : img_class}, ignore_index = True)

KeyboardInterrupt: 

In [ ]:
images_expanded.to_csv('words_expande.csv')

In [9]:
data_dir = pathlib.Path('../data/tiny-imagenet-200')
image_count = len(list(data_dir.glob('**/*.JPEG')))

In [10]:
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
sum([cls in images['Id'].unique() for cls in CLASS_NAMES])

101

## Testing Inception v3 out of the box

In [11]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


In [12]:
data_dir = data_dir
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception_v3"
# Number of classes in the dataset
num_classes = 200
# Batch size for training (change depending on how much memory you have)
batch_size = 8
# Number of epochs to train for
num_epochs = 3
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = True

In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)


In [ ]:
#change last layers in model to match tiny imagenet
model.AuxLogits.fc = nn.Linear(768, num_classes)
model.fc = nn.Linear(2048, num_classes)

In [ ]:
#inception expects input size 3*299*299
input_size = 299

In [ ]:
from PIL import Image

In [ ]:
#test on one image
input_image_one = Image.open("../data/tiny-imagenet-200/train/n01443537/images/n01443537_0.JPEG")

preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#converts image to 3*299*299
input_tensor_one = preprocess(input_image_one)

#convert to batches
input_batch = torch.cat((input_tensor_one.unsqueeze(0), input_tensor_one.unsqueeze(0)))

with torch.no_grad():
    output = model(input_batch)

In [ ]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

In [ ]:
torch.topk(probabilities, 5)

In [ ]:
labels = pd.read_csv("../data/tiny-imagenet-200/wnids.txt", names = ['class_labels'], sep = '\t')
labels['predictions'] = probabilities

In [ ]:
labels.sort_values(by = ['predictions'], ascending = False)

### Updating feature extraction to match our dataset

In [22]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=1, is_inception=True):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training (no val for now)
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [14]:
def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [15]:
model_ft = models.inception_v3(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extract)
# Handle the auxilary net
num_ftrs = model_ft.AuxLogits.fc.in_features
model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
# Handle the primary net
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes)
input_size = 299

#### Train on tiny imagenet training images

In [16]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train']}

In [17]:
params_to_update = []
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [ ]:
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=True)